# Applying Machine Learning on rest brain imaging dataset

        1. Using BidsGrabber to understand the data layout
        2. Preparing data matrix from 4D to 2D
        3. Preparing condition array 0f 1D, which stores the respective group associated with each subject_id
        
        For machine leaning part, we are using scikit learn library
        using the link [http://nilearn.github.io/decoding/decoding_intro.html#loading-and-preparing-the-data]
        
        4. Applying SVM Classifier without selecting any voxles and noting the accuracy.
        5. Applying Recursive Feature Extraction algoritm using SVM and then check the classification accuracy.

#### setting config

    setting parameteres values here, so if we need to change value anytime, we can make the changes here, no need to find in the pipeline or functions 

In [1]:
data_input_dir = '/data/NYU/'
subjects_info='/data/NYU/participants.tsv'
data_output_dir = '/output/'
no_of_subjects = 12 #nof of subjects to consider for computation

#### BidsLayout to easily access the structure of files and filenames

In [2]:
from bids.grabbids import BIDSLayout

In [3]:
layout = BIDSLayout(data_input_dir)

In [4]:
!tree $data_input_dir

/data/NYU/
|-- T1w.json
|-- participants.tsv
|-- sub-0050952
|   |-- sub-0050952_T1w.nii.gz
|   `-- sub-0050952_task-rest_run-1_bold.nii.gz
|-- sub-0051159
|   |-- sub-0051159_T1w.nii.gz
|   `-- sub-0051159_task-rest_run-1_bold.nii.gz
`-- task-rest_bold.json

2 directories, 7 files


In [5]:
subject_list = (layout.get_subjects())[0:(no_of_subjects)] #this gives the list of subjects in a given directory
subject_list

['0050952', '0051159']

#### function to get the nifti filenames

In [6]:
def get_nifti_filenames(subject_id,data_input_dir):
#     Remember that all the necesary imports need to be INSIDE the function for the Function Interface to work!
    from bids.grabbids import BIDSLayout
    
    layout = BIDSLayout(data_input_dir)
    
    func_file_path = [f.filename for f in layout.get(subject=subject_id, type='bold', extensions=['nii', 'nii.gz'])]
    
    return func_file_path[0]

#### Preparing a list of filenames

In [7]:
func_filenames = list()
for i , subject_id in enumerate (subject_list):
    func_filenames.append(get_nifti_filenames(subject_id, data_input_dir)) 
                        

#### showing all the nifti file name with path

In [8]:
#nifti_images_list
func_filenames

['/data/NYU/sub-0050952/sub-0050952_task-rest_run-1_bold.nii.gz',
 '/data/NYU/sub-0051159/sub-0051159_task-rest_run-1_bold.nii.gz']

####  FROM 4-DIMENSIONAL IMAGES TO 2-DIMENSIONAL ARRAY: MASKING
    Neuroimaging data are represented in 4 dimensions: 3 spatial dimensions, and one dimension to index time or trials.
   
    Machine leanrning algorithms,on the other hand, only accept 2-dimensional samples × features matrices
   
    Depending on the setting, voxels and time series can be considered as features or samples.
   
    The selected voxels form a brain mask. Such a mask is often given along with the datasets or can be computed with software tools such as FSL or SPM. 
   

In [9]:
from nilearn.input_data import NiftiMasker,MultiNiftiMasker

Converting 4d array into 2d array using reshape of numpy

In [10]:
#test_imgs = list()
#test_imgs.append('/data/NYU/sub-0050952/func/sub-0050952_task-rest_run-1_bold.nii.gz')
#test_imgs.append('/data/NYU/sub-0050953/func/sub-0050953_task-rest_run-1_bold.nii.gz',)

#img size is 64*80*33
multi_nifti_masker = MultiNiftiMasker(standardize=True)
fmri_masked = multi_nifti_masker.fit_transform(func_filenames) 
# fmri_masked = list of ndarray of each subject and each ndarray represents timeseries * selected voxels from brain mask

#### DATA PREPARATION: 
    For the machine learning settings, we need a data matrix, that we will denote X, and optionally a target    variable to predict, y
   
    X = timeseries * voxels

In [11]:
import numpy as np
X = np.concatenate(fmri_masked, axis=0 )

In [12]:
X.shape

(352, 230486)

#### loading subjects meta data and will form the (subject_id - group)
  With pandas library , we load tsv file into data frame , and then select the selected columns form this

In [13]:
import pandas as pd

subjects_meta_data=pd.read_csv(subjects_info,sep='\t')

#### extracting subject_id, group that is given in subject_list
   1. preparing mask to select the selected subject_id
   2. using that mask to select the participant_id , dx_group
   
 In participants.tsv file, we have many variables, however we are using participant_id and group

In [14]:
subject_id = 'participant_id'
group = 'dx_group' # given in file

mask=subjects_meta_data[subject_id].isin(subject_list) # check setting config for subject_id
subject_id_group = subjects_meta_data[mask][[subject_id,group]] # check setting config for group

In [15]:
subject_id_group

,participant_id,dx_group
0,50952,Autism
183,51159,Control


#### there are two groups, Autism  , Control

In [21]:
groups = subject_id_group['dx_group'].values
type(groups)

numpy.ndarray

In [22]:
groups

array(['Autism', 'Control'], dtype=object)

Function to create a labels set with respective no of time points for each subject

Setting labels `1` as `Autism` and `0` as `Control`

In [23]:
def labels_set(nifti_files_path,subjects_groups):
    
    import nibabel as nb
    import numpy as np
    labels=[]
    
    for i,img in enumerate(nifti_files_path):
        img_load = nb.load(img)
        labels += img_load.shape[3] * [(1 if subjects_groups[i] =='Autism' else 0)]
        
    return np.asarray(labels)
   

Now, we call labels_set , passing parameters values of list of images path and groups_id

In [24]:
labels=labels_set(func_filenames,groups)
#labels=np.transpose(np.matrix(labels))
labels.shape

(352,)

In [25]:
labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,

#### Applying classifier , train and test 


   we use a Support Vector Classification, with a linear kernel.
   1.The svc object is an object that can be fit (or trained) on data with labels, and then predict labels on data without.

In [26]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')
print(svc)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


####  Measuring prediction scores using cross-validation
   1. The proper way to measure error rates or prediction accuracy is via cross-validation: leaving out some data and testing on it.

Scikit-learn has tools to perform cross-validation easier:
You can speed up the computation by using n_jobs=-1, which will spread the computation equally across all processors (but might not work under Windows):

#### This is just for testing, as on my system, memory issue was showing, so just want to check the cross validation

In [27]:
#X1 = np.array([[1, 2, 3], [4, 5, 6],[4, 5, 6],[4, 5, 6],[1, 2, 3],[1, 2, 3]], np.int32)
#y = np.array([[1],[2],[2],[2],[1],[1]])
#y=y.reshape(6,)

#### This is without using any feature extraction 

In [28]:
from sklearn.cross_validation import cross_val_score
cv_scores = cross_val_score(svc, X, labels, cv=2, n_jobs=2, verbose=10)
print(cv_scores)


/opt/conda/envs/neuro/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


OSError: [Errno 28] No space left on device

#### This is using with the feature extraction algroithm

 Recursive feature elimination with cross-validation


A recursive feature elimination example with automatic tuning of the
number of features selected with cross-validation.


In [23]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

# Create the RFE object and compute a cross-validated score.
svc = SVC(kernel="linear")
# The "accuracy" scoring is proportional to the number of correct
# classifications
#If step is greater than or equal to 1, then step corresponds to the (integer) number of features 
# to remove at each iteration. 

rfecv = RFECV(estimator=svc, step=3, cv=StratifiedKFold(2),
              scoring='accuracy')

rfecv.fit(X, labels)

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

ValueError: The number of classes has to be greater than one; got 1